In [36]:
from dragon import dragonV
import numpy as np
import torch

## data processing

In [14]:
input_path = '/Users/ivory/Documents/github/gcdn_n/run_net/input/fast_rear.xlsx'
output_path = '/Users/ivory/Documents/github/gcdn_n/run_net/output/'
seleted_openpose_joint_idx_list = [8, 9, 10, 11, 12, 13, 14, 19, 20, 21, 22, 23, 24]

In [18]:
frame_list = dragonV.xlsx2data(input_path)
#normalization
norm_frame_list =  dragonV.nomalize_data(frame_list)
#select lower joint
selected_norm_frame_data_list = dragonV.get_selected_joint_pos_frame_list(norm_frame_list, seleted_openpose_joint_idx_list)

In [17]:
len(selected_norm_frame_data_list)

207

In [21]:
np.array(selected_norm_frame_data_list)

array([[0.99997877, 0.99977087, 0.99982422, ..., 1.        , 0.99366179,
        1.        ],
       [0.99997877, 0.99977087, 0.99982422, ..., 1.        , 0.99366179,
        1.        ],
       [0.99997877, 0.99997454, 0.99982422, ..., 1.        , 0.99361823,
        1.        ],
       ...,
       [0.00607244, 0.02661326, 0.0285346 , ..., 0.66846091, 0.00659958,
        0.01616173],
       [0.00596628, 0.02583251, 0.02839788, ..., 0.66578815, 0.01280711,
        0.00812195],
       [0.        , 0.02723276, 0.        , ..., 0.66302423, 0.        ,
        0.        ]])

In [28]:
def sliding_window(input_list, window_size):
    return [input_list[i:i + window_size] for i in range(len(input_list) - window_size + 1)]

input_features = sliding_window(selected_norm_frame_data_list, 9)


In [35]:
print('samples: ',len(input_features))
print('timesteps: ',len(input_features[0]))
print('features: ',len(input_features[0][0]))

samples:  199
timesteps:  9
features:  26


In [57]:
input_data_torch = torch.from_numpy((np.array(input_features)))
print(input_data_torch.shape)
input_data_torch = input_data_torch.to(torch.float32)

torch.Size([199, 9, 26])


In [59]:
input_data_torch[0][0]

tensor([1.0000, 0.9998, 0.9998, 0.9995, 0.9999, 0.9886, 0.9936, 1.0000, 1.0000,
        0.9995, 0.9934, 0.9993, 0.9999, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.9856, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.9937, 1.0000])

In [43]:
has_nan = torch.isnan(input_data_torch).any().item()
print("NaN 값이 있는지 여부:", has_nan)

NaN 값이 있는지 여부: False


## NetWork Part

In [45]:
import torch
import torch.nn as nn

In [46]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTM, self).__init__()
        
        self.hidden_size = hidden_size
        
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        _, (hidden, _) = self.lstm(x)
        out = self.fc(hidden[-1])
        out = self.sigmoid(out)
        return out

In [47]:
input_size = 26
output_size = 4
hidden_size = 128

In [48]:
model = LSTM(input_size, hidden_size, output_size)

In [49]:
model_weights = torch.load('../models/100_epochs.pth', map_location=torch.device('cpu'))
model.load_state_dict(model_weights)
model.eval()

LSTM(
  (lstm): LSTM(26, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=4, bias=True)
  (sigmoid): Sigmoid()
)

In [107]:
output = model(input_data_torch)

In [108]:
predicted = (output > 0.5).int()
predicted_np =  predicted.numpy()

In [109]:
margin_np = np.array([[0,0,0,0], [0,0,0,0], [0,0,0,0], [0,0,0,0]])
m_predicted_np = np.append(margin_np, predicted_np, axis=0)
m_predicted_np = np.append(m_predicted_np, margin_np, axis=0)


In [111]:
len(m_predicted_np)

207

In [112]:
np.save(output_path + '/predicted.npy', m_predicted_np)